In [6]:
#network = "mainnet"
network = "polygon"

import sys
import json
from collections import Counter
sys.path.append('../')

from helper import FunctionName as fn

umbra_contract_txs_json = f"../umbra/data/{network}/umbra_contract_txs.json"
skr_contract_txs_json = f"../umbra/data/{network}/stealth_key_registry_contract_txs.json"

with open(umbra_contract_txs_json, "r") as file:
    data = json.load(file)

with open(skr_contract_txs_json, "r") as file:
    skr_data = json.load(file)

contract_txs = data["result"]
skr_contract_txs = skr_data["result"]

In [5]:
"""
    ### common_statistics --> ###
    #1
    Are there stealth addresses that have been used more than once?
"""

stealths = []

for d in contract_txs:
    # these stealths occure twice because of send
    if d["functionName"] == fn.W_TOKEN.value:
        continue

    receiver = d[d["functionName"]]["_receiver"]
    stealths.append(receiver)

stealths = Counter(stealths)

print(list(filter(lambda d: stealths[d] > 1, stealths)))

"""
    The answer is no as expected, but we had to be sure.
"""
pass

[]


In [ ]:
"""
    ### common_statistics --> ###
    #2
    To check wether the whole eth has been withdrawn, we need the to check the sending amount
    since this is the only way to figure out how much eth was stored on the stealth account. 
"""

for d in contract_txs:
    if d["functionName"] == fn.S_ETH.value:
        sent_eth = int(d["value"])

        receiver = d[d["functionName"]]["_receiver"]
        for tx in d[d["functionName"]][receiver]:
            withdrawn_eth = int(tx["value"]) + (int(tx["gasUsed"]) * int(tx["gasPrice"]))

            if sent_eth != withdrawn_eth:
                s = sent_eth/pow(10, 18)
                w = withdrawn_eth/pow(10, 18)
                print(tx["from"])
        

"""
    At the time of writing this comment (2022.12.01.) there are 3 case when the compared two amounts
    are not equal:

        0x9f6f037b8782c04922eb7e1a5f602f32279e17cf
        0x07ebf2789f239d56b936b37a8d753a3429ebeb62
        0xc7f6aec4f08b1ec6f220ab39e6a590909e76195c
    
    Checking these addresses on etherscan.io, we can see that the unspent amount is still stored on the
    stealth account in case of the last two. In case of the first one (0x9f6f037b8782c04922eb7e1a5f602f32279e17cf),
    I have no idea where that little eth have gone, because etherscan shows that this account has 0 eth on it.

    These txs can diminish all of our heuristics, since if not the whole amount was spent, then there's a higher chance
    that it was really a tx where someone payed to someone else. Also, if there are more withdrawal transactions from a
    stealth address, how could we know who is the real recipient or if more address has communicated with the
    stealth key registry, which one should we look after for?
    For future sustainability, we will implement this checking module into the instantiator class of the heuristics
    (MyStatistics).

    ###
    ###
    ###

    Since the other txs transfer all the eth/token from stealth addresses, we can be
    basically 100% sure about that if the recepient of the tx is an address which has
    communicated with the skr, then the recepient is really the person
    who controls the stealth address, not just another person who has communicated with the skr.
    We can say it because it is highly unlikely that someone has to pay exactly the same amount to
    someone else that she/he has received.
"""
pass

In [ ]:
# Check if there is an address who received both eth and token

from collections import Counter

with open("../umbra/data/umbra_contract_txs.json", "r") as file:
    data = json.load(file)

contract_txs = data["result"]

token = []
eth = []

for d in contract_txs:
    if d["functionName"] == fn.S_ETH.value:
        receiver = d[d["functionName"]]["_receiver"]

        for tx in d[d["functionName"]][receiver]:
            eth.append(tx["to"])

    if d["functionName"] == fn.W_TOKEN.value:
        acceptor = d[d["functionName"]]["_acceptor"]
        token.append(acceptor)


for t in token:
    if t in eth:
        print(t)

# Since there's nothing to print, no one has used umbra to withdraw both eth and token

token = Counter(token)
eth = Counter(eth)

print(max(dict(token).values()))
print(max(dict(eth).values()))

# But there are people who have used an address to withdraw more than once exclusively for eth or token

In [2]:
txs_to_contract = []

for c in contract_txs:
    txs_to_contract.append(c["from"])

adresses_of_txs_to_contract_count = Counter(txs_to_contract) #count of txs grouped by address

with open(f"./results/{network}/sender_addresses.json", "w") as file:
    json.dump(dict(adresses_of_txs_to_contract_count), file)

In [7]:
registrants = set()

for d in skr_contract_txs:
    registrants.add(d["from"])

print(len(registrants))

22702
